In [1]:
import pandas
import os
import glob

In [2]:
folder = os.path.expanduser('~/localdatalake/tse_refined')

In [3]:
files = glob.glob(os.path.join(folder, 'votos', 'VotoSecaoDetalhe*.csv.gz'))

In [4]:
df = pandas.concat((pandas.read_csv(x, sep=';') for x in files), ignore_index=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10854876 entries, 0 to 10854875
Data columns (total 17 columns):
Ano                     int64
Turno                   int64
Eleição_nome            object
UF                      object
UE                      object
Cargo                   int64
Município               int64
Zona                    int64
Votos_aptos             int64
Votos_comparecimento    int64
Votos_abstenções        int64
Votos_nominais          int64
Votos_brancos           int64
Votos_nulos             int64
Votos_legenda           int64
Votos_pendentes         int64
Seção                   int64
dtypes: int64(14), object(3)
memory usage: 1.4+ GB


In [6]:
df.head()

,Ano,Turno,Eleição_nome,UF,UE,Cargo,Município,Zona,Votos_aptos,Votos_comparecimento,Votos_abstenções,Votos_nominais,Votos_brancos,Votos_nulos,Votos_legenda,Votos_pendentes,Seção
0,2012,1,ELEIÇÃO MUNICIPAL 2012,ES,56014,11,56014,8,359,289,70,266,8,15,0,0,1
1,2012,1,ELEIÇÃO MUNICIPAL 2012,ES,56014,13,56014,8,359,289,70,255,4,3,27,0,1
2,2012,1,ELEIÇÃO MUNICIPAL 2012,ES,56014,11,56014,8,364,277,87,260,7,10,0,0,2
3,2012,1,ELEIÇÃO MUNICIPAL 2012,ES,56014,13,56014,8,364,277,87,241,6,7,23,0,2
4,2012,1,ELEIÇÃO MUNICIPAL 2012,ES,56014,11,56014,8,360,287,73,271,3,13,0,0,3


In [7]:
df['Ano'].value_counts().sort_index()

2008     818448
2009        674
2010    2456006
2011        476
2012     901271
2014    2719583
2016     956455
2018    3001963
Name: Ano, dtype: int64

In [8]:
df['Eleição_nome'].value_counts().sort_index()

ELEIÇÃO GERAL FEDERAL 2018         908898
ELEIÇÃO MUNICIPAL 2012             901271
ELEIÇÕES 2008                      811156
ELEIÇÕES 2010                     2456006
ELEIÇÕES GERAIS 2014              2719583
ELEIÇÕES GERAIS ESTADUAIS 2018    2093065
ELEIÇÕES MUNICIPAIS 2016           956455
ELEIÇÕES SUPLEMENTARES                 78
ELEIÇÕES SUPLEMENTARES 2008          7214
PLEBISCITO                           1150
Name: Eleição_nome, dtype: int64

In [9]:
from sklearn.decomposition import PCA
pca = PCA()

In [10]:
cols = [x for x in df.columns if 'Votos' in x]
for col in cols:
    df = df[df[col]>=0]


In [44]:
pca.fit(df[df['Ano']>=2018][cols])

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [45]:
print(pca.explained_variance_ratio_)

[7.33396061e-01 1.76233251e-01 4.67031807e-02 3.56054004e-02
 5.48181239e-03 2.54203163e-03 3.82630794e-05 2.27997495e-32]


In [46]:
import numpy as np
np.cumsum((pca.explained_variance_ratio_))

array([0.73339606, 0.90962931, 0.95633249, 0.99193789, 0.99741971,
       0.99996174, 1.        , 1.        ])

In [47]:
vars(pca).keys()

dict_keys(['n_components', 'copy', 'whiten', 'svd_solver', 'tol', 'iterated_power', 'random_state', '_fit_svd_solver', 'mean_', 'noise_variance_', 'n_samples_', 'n_features_', 'components_', 'n_components_', 'explained_variance_', 'explained_variance_ratio_', 'singular_values_'])

In [48]:
pca.components_[-1]

array([-5.77350269e-01,  5.77350269e-01,  5.77350269e-01,  0.00000000e+00,
       -2.08166817e-17, -4.16333634e-17,  2.08166817e-17,  7.48438312e-18])

In [49]:
tt = df[df['Ano']>=2018]
(tt['Votos_aptos']-tt['Votos_comparecimento']-tt['Votos_abstenções']).value_counts()

0    3001963
dtype: int64

In [50]:
tt[(tt['Votos_aptos']-tt['Votos_comparecimento']-tt['Votos_abstenções'])!=0]

,Ano,Turno,Eleição_nome,UF,UE,Cargo,Município,Zona,Votos_aptos,Votos_comparecimento,Votos_abstenções,Votos_nominais,Votos_brancos,Votos_nulos,Votos_legenda,Votos_pendentes,Seção


In [51]:
[x if abs(x)>1e-3 else 0 for x in pca.components_[-1]]

[-0.5773502691896261, 0.5773502691896257, 0.5773502691896253, 0, 0, 0, 0, 0]

In [58]:
cols = [x for x in df.columns if 'Votos' in x and ('absten' not in x) and ('aptos' not in x) and ('pendente' not in x)]
print(cols)
print('\n')
pca.fit(df[(df['Ano']>=2018)&(df['Cargo'].isin([6,7,8]))][cols])
for i in range(1,6):
    print("Variance_ratio: {:.5f}".format(pca.explained_variance_ratio_[-i]))
    print(', '.join(["{:+.4f}".format(x) if abs(x)>1e-4 else "0" for x in pca.components_[-i]]))
print('\n')
pca.fit(df[(df['Ano']>=2018)&~(df['Cargo'].isin([6,7,8]))][cols])
for i in range(1,6):
    print("Variance_ratio: {:.5f}".format(pca.explained_variance_ratio_[-i]))
    print(', '.join(["{:+.4f}".format(x) if abs(x)>1e-4 else "0" for x in pca.components_[-i]]))

['Votos_comparecimento', 'Votos_nominais', 'Votos_brancos', 'Votos_nulos', 'Votos_legenda']


Variance_ratio: 0.00002
+0.4470, -0.4469, -0.4475, -0.4469, -0.4477
Variance_ratio: 0.00731
-0.0298, +0.0360, -0.8358, +0.4208, +0.3495
Variance_ratio: 0.00824
-0.1536, +0.1779, -0.0558, +0.5353, -0.8094
Variance_ratio: 0.05791
+0.3993, -0.6284, +0.3083, +0.5742, +0.1446
Variance_ratio: 0.92652
-0.7850, -0.6103, -0.0553, -0.0832, -0.0363


Variance_ratio: 0.00000
0, 0, 0, 0, +1.0000
Variance_ratio: 0.00650
-0.0419, +0.0481, -0.9177, +0.3921, 0
Variance_ratio: 0.05859
+0.2600, -0.3864, +0.3203, +0.8249, 0
Variance_ratio: 0.14144
+0.8454, -0.3476, -0.2060, -0.3493, 0
Variance_ratio: 0.79347
+0.4647, +0.8530, +0.1129, +0.2093, 0


0      3958398
383          4
363          4
766          1
726          1
Name: Votos_pendentes, dtype: int64

In [23]:
tt[(tt['Votos_pendentes'])!=0]

,Ano,Turno,Eleição_nome,UF,UE,Cargo,Município,Zona,Votos_aptos,Votos_comparecimento,Votos_abstenções,Votos_nominais,Votos_brancos,Votos_nulos,Votos_legenda,Votos_pendentes,Seção
8281237,2018,1,ELEIÇÕES GERAIS ESTADUAIS 2018,BA,BA,5,38490,18,476,383,93,0,0,0,0,766,328
8305338,2018,1,ELEIÇÕES GERAIS ESTADUAIS 2018,BA,BA,3,38490,18,384,363,21,0,0,0,0,363,202
8308089,2018,1,ELEIÇÕES GERAIS ESTADUAIS 2018,BA,BA,7,38490,18,384,363,21,0,0,0,0,363,202
8313198,2018,1,ELEIÇÕES GERAIS ESTADUAIS 2018,BA,BA,3,38490,18,476,383,93,0,0,0,0,383,328
8315024,2018,1,ELEIÇÕES GERAIS ESTADUAIS 2018,BA,BA,6,38490,18,476,383,93,0,0,0,0,383,328
8322557,2018,1,ELEIÇÕES GERAIS ESTADUAIS 2018,BA,BA,5,38490,18,384,363,21,0,0,0,0,726,202
8360379,2018,1,ELEIÇÕES GERAIS ESTADUAIS 2018,BA,BA,6,38490,18,384,363,21,0,0,0,0,363,202
8369542,2018,1,ELEIÇÕES GERAIS ESTADUAIS 2018,BA,BA,7,38490,18,476,383,93,0,0,0,0,383,328
9069061,2018,1,ELEIÇÃO GERAL FEDERAL 2018,BA,BR,1,38490,18,476,383,93,0,0,0,0,383,328
9224210,2018,1,ELEIÇÃO GERAL FEDERAL 2018,BA,BR,1,38490,18,384,363,21,0,0,0,0,363,202


In [24]:
[x if abs(x)>1e-3 else 0 for x in pca.components_[-3]]

[-0.019388055763604,
 -0.03579775690060701,
 0.016409701137002998,
 0.054808873113231654,
 -0.5896127057909047,
 0.2583944502993008,
 0.7620093464153334,
 0]

In [27]:
(tt['Votos_comparecimento']-tt['Votos_nominais']-tt['Votos_brancos']-tt['Votos_nulos']-tt['Votos_legenda']).value_counts() / len(tt)

 0      8.853830e-01
-285    8.920253e-04
-284    8.854570e-04
-295    8.831833e-04
-286    8.816676e-04
            ...     
-452    2.526268e-07
-10     2.526268e-07
-2      2.526268e-07
-448    2.526268e-07
-3      2.526268e-07
Length: 462, dtype: float64

In [28]:
cols

['Votos_aptos',
 'Votos_comparecimento',
 'Votos_abstenções',
 'Votos_nominais',
 'Votos_brancos',
 'Votos_nulos',
 'Votos_legenda',
 'Votos_pendentes']

In [ ]:
pca_exla